## Resource-Constrained Project Scheduling Problem with Sequence-Dependent Setup Time

This notebook demonstrates how to model and solve the Resource-Constrained Project Scheduling Problem with Sequence-Dependent Setup Times
using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API.
This problem extends the classical RCPSP (see [`rcpsp.ipynb`](https://github.com/radovluk/CP_Cookbook/blob/main/notebooks/rcpsp.ipynb)).

### Problem Definiton

In the RCPSP with Sequence-Dependent Setup Times, we are given a finite set of tasks indexed by $i \in \{1,\dots,N\}$ and a finite set of **unary (disjunctive)** resources indexed by $k \in \{1,\dots,M\}$. Each task $i$ has a fixed duration $PT_i>0$. For each unary resource $k$, let $I_k \subseteq \{1,\dots,N\}$ denote the tasks that are processed on $k$ one at a time, and let $\mathrm{TM}_{k}=[S^{(k)}{ij}]$ be a transition (setup) matrix giving the setup time incurred if task $j$ immediately follows task $i$ on $k$. A set of precedence relations $P \subseteq \{1,\dots,N\}^2$ specifies required ordering (i.e., $(i,j)\in P$ means task $i$ must precede task $j$). The objective is to minimize the makespan $C_{\max}$ — the completion time of the last finishing task — while satisfying both the precedence constraints and, on each unary resource, non-overlap together with the sequence-dependent setup delays specified by $\mathrm{TM}_k$.

### CPLEX Formulation

$$
\begin{aligned}
\min\quad
& \max_{i \in [1..N]} \mathrm{endOf}(x_i)
& &
& \text{(1)} \\
\text{s.t.}\quad
& \mathrm{endBeforeStart}(x_i, x_j)
& &
\forall (i,j) \in P
& \text{(2)} \\
&
\mathrm{noOverlap}\!\bigl(\mathrm{seq}_k,\ \mathrm{TM}_k\bigr)
& &
\forall k \in [1..M]
& \text{(3)} \\
&
\text{interval } x_i,\ \text{size} = PT_i
& &
\forall i \in [1..N]
& \text{(4a)} \\
&
\text{sequence } \mathrm{seq}_k\ \text{over }\{x_i \mid i \in I_k\}
& &
\forall k \in [1..M]
& \text{(4b)}
\end{aligned}
$$


Objective:
- **(1)** The **makespan** $C_{\max}$ — the completion time of the last finishing task — is **minimized**.

Modeling constraints:
- **(2)** All precedence relations $(i,j)\in P$ are respected.  
- **(3)** For each unary resource $k$, tasks in $I_k$ are processed **without overlap** and with **sequence-dependent setup times** given by $\mathrm{TM}_k$ (i.e., if $j$ follows $i$ on $r$ then $\mathrm{startOf}(x_j)\ge \mathrm{endOf}(x_i)+S^{(k)}_{ij}$).

Variables:
- **(4a)** Interval variables $x_i$ (one per task) with fixed size $PT_i$, deciding task start/end times.
- **(4b)** For each unary resource $k$, a sequence variable $\mathrm{seq}_k$ that orders $\{x_i \mid i\in I_k\}$ and applies the transition matrix $\mathrm{TM}_k$.

#### Symbols and Notation

| Symbol / Function | Meaning | docplex.cp reference |
|---|---|---|
| $N$ | Number of tasks | — |
| $M$ | Number of unary resources | — |
| $i$ | Task index ($i \in [1..N]$) | — |
| $k$ | Unary resource index ($k \in [1..M]$) | — |
| $PT_i$ | Processing time (duration) of task $i$ | — |
| $P$ | Precedence set $\subseteq [1..N]\times[1..N]$ | — |
| $I_k$ | Set of tasks processed on unary resource $k$ | — |
| $x_i$ | Interval variable for task $i$ (size $PT_i$) | [interval_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html#docplex.cp.expression.interval_var) |
| $\mathrm{seq}_k$ | Sequence variable defining the order of tasks on resource $k$ | [sequence_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html#docplex.cp.expression.sequence_var) |
| $\mathrm{TM}_k$ | Transition (setup) matrix on $k$ | [transition_matrix](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.transition_matrix) |
| $\mathrm{endBeforeStart}(x_i,x_j)$ | Enforces precedence $i \rightarrow j$ | [end_before_start](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_before_start) |
| $\mathrm{noOverlap}(\mathrm{seq}_k,\mathrm{TM}_k)$ | Enforces non-overlap and setup times on unary resource $k$ | [no_overlap](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.no_overlap) |
| $\mathrm{endOf}(x_i)$ | End time of task $i$ | [end_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_of) |
| $\min \max_i \mathrm{endOf}(x_i)$ | Makespan minimization objective | [minimize](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.minimize) |


### Aditional Resources

- **IBM Modeling sequence-dependent setup times**
  - https://www.ibm.com/docs/en/icos/22.1.1?topic=models-modeling-sequence-dependent-setup-times